In [18]:
import s3fs
import json

In [26]:
with open("creodias-credentials-public.json") as f:
    credentials = json.load(f)
    s3 = s3fs.S3FileSystem(
        anon=False, 
        client_kwargs=dict(endpoint_url="https://data.cloudferro.com"), 
        **credentials
    )

In [27]:
s3.listdir("EODATA")

ConnectTimeoutError: Connect timeout on endpoint URL: "https://data.cloudferro.com/EODATA?list-type=2&prefix=&delimiter=%2F&encoding-type=url"